In [1]:
import numpy as np
import cvxpy as cp

In [2]:
# Problem Data from https://github.com/cvxgrp/cvxbook_additional_exercises/blob/main/python/uav_design_data.py
K = 5    # number of missions
D = np.array([1.2e5, 4.5e5, 5.2e5, 2.4e5, 1.1e6])      # m
V = np.array([8.2e1, 5.5e1, 6.5e1, 7e1, 4.8e1])        # m/s
W_pay = np.array([2.5e1, 3.2e1, 3.2e1, 5.7e1, 9.3e0])  # kg

W_eng_min = 5      # kg
W_eng_max = 50     # kg
W_bat_min = 10     # kg
W_bat_max = 100    # kg
S_min = 8          # m^2
S_max = 30         # m^2
alpha_max = 6      # degrees

# Used specs from the below links for reference.
# (1) https://physicsworld.com/a/lithium-ion-batteries-break-energy-density-record/
# (2) https://en.wikipedia.org/wiki/Power-to-weight_ratio#Electric_motors_and_electromotive_generators
# (3) https://www2.eecs.berkeley.edu/Pubs/TechRpts/2015/EECS-2015-22.pdf
# (4) https://en.wikipedia.org/wiki/Angle_of_attack#/media/File:Coefficients_of_Drag_and_Lift_vs_AOA.jpg
# (5) https://en.wikipedia.org/wiki/Boeing_747#Specifications
W_base = 100.   # kg
CW = 5.         # dimensionless    (5)
cL = 0.08       # dimensionless    (4)
cD0 = 0.0005    # dimensionless    (4)
cD1 = 0.02      # dimensionless    (4)
CP = 0.002      # dimensionless    (2)
CE = 2.5e6      # J/kg             (1)
rho = 0.91      # kg/m^3           (3)

In [3]:
def C_L(alpha):
    return cL*alpha

def C_D(alpha):
    return cD1 + cD0*alpha**2

In [4]:
### optimization variables ###
W_eng = cp.Variable(1, pos=True)
W_bat = cp.Variable(1, pos=True)
S = cp.Variable(1, pos=True)
P = cp.Variable(K, pos=True)
alpha = cp.Variable(K, pos=True)

### derived expressions ###
W_wing = CW*S**(1.2)
W_k = [W_bat + W_eng + W_wing + W_base + W_pay[k] for k in range(K)]
P_max = ((1/CP)**(1/0.803)) * (W_eng**(1/0.803))
F_k_lift = [0.5*rho* V[k]**2 * C_L(alpha[k]) * S for k in range(K)]
F_k_drag = [0.5*rho * V[k]**2 * C_D(alpha[k])*S for k in range(K)]
T_k = [P[k] * (1/V[k]) for k in range(K)]
E = CE*W_bat
C_des = 100*W_eng + 45*W_bat + 2 * W_wing
C_mis = cp.sum([T_k[k] + 10*alpha[k] for k in range(K)])
### ###

### constraints and objective ###
constraints = [W_eng_min <= W_eng, W_eng <= W_eng_max,
               W_bat_min <= W_bat, W_bat <= W_bat_max,
               S_min <= S, S <= S_max]
constraints += [constr for constr_tuple in [(P[k] <= P_max,
                 alpha[k] <= alpha_max) for k in range(K)]
                 for constr in constr_tuple]
constraints += [F_k_lift[k] >= W_k[k] for k in range(K)]
constraints += [F_k_drag[k] <= T_k[k] for k in range(K)]
constraints += [P[k] * D[k] * (1/V[k]) <= E for k in range(K)]

obj = cp.Minimize(C_des + C_mis)
prob = cp.Problem(obj, constraints)
# print(prob.is_dgp()) # useful for debugging: comment out constraint blocks and run this
prob.solve(gp=True)

/Users/q3/opt/anaconda3/envs/boyd_linear-algebra/lib/python3.11/site-packages/cvxpy/reductions/solvers/solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


6005.911367450539

In [5]:
print(f'Design Cost: {C_des.value}')
print(f'Mission Cost: {C_mis.value}')

Design Cost: [4449.32439313]
Mission Cost: 1556.586974325188


In [18]:
for k in range(K):
    print(np.abs(F_k_drag[k][0].value - T_k[k].value) < 1e-5)

True
True
True
True
True


In [21]:
for k in range(K):
    print(np.abs(F_k_lift[k][0].value - W_k[k].value) < 1e-2)

[ True]
[ True]
[ True]
[ True]
[ True]


In [13]:
for drag in F_k_drag:
    print(drag[0].value)

489.73928888082395
220.76301919551952
307.9687903368573
357.11855911547593
168.33194244245712


In [25]:
for tk in T_k:
    print(tk.value)

489.739289540237
220.76302847287894
307.96879910517447
357.1185675911295
168.33194213679587


In [23]:
for lift in F_k_lift:
    print(lift[0].value)

269.6478752165577
276.6470324215976
276.64758234779765
301.64783993821214
253.94584559160754


In [22]:
for weight in W_k:
    print(weight[0].value)

269.645663356979
276.645663356979
276.645663356979
301.645663356979
253.94566335697903


In [6]:
for p_k in P:
    print(p_k.value)

40158.62174229943
12141.966566008343
20017.97194183634
24998.299731379066
8079.933222566203


In [8]:
for a_k in alpha:
    print(a_k.value)

0.13771394751546792
0.3140575703917646
0.2248582340634128
0.2114037901856076
0.3785012057409691


In [9]:
W_eng.value

array([9.95094608])

In [10]:
W_bat.value

array([74.06605467])

In [11]:
S.value

array([7.99999999])